In [1]:
import numpy as np
import pandas as pd

import nltk
nltk.download("stopwords")

data = pd.read_csv("questions.tsv",delimiter='\t',quoting=3)

[nltk_data] Downloading package stopwords to /home/vipul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data.shape

(38, 2)

In [3]:
data.tail(10)

,que,label
28,your email id,contact
29,How can i meet you,contact
30,Where can i meet you,address
31,Your address?,address
32,What's the location of your office,address
33,Where to meet you?,address
34,your loaction,address
35,Hey,greeting
36,goodbye,goodbye
37,hy,greeting


In [4]:
from sklearn.utils import shuffle
data = shuffle(data)
data.reset_index(drop=True,inplace=True)

In [5]:
data.head()

,que,label
0,what is your name,name
1,Hey,greeting
2,What services does your firm provide?,services
3,cya,goodbye
4,what are your services?,services


In [6]:
import re

In [7]:
data['que'][15]

'Is anyone there?'

In [8]:
que = re.sub('[^a-zA-Z]', ' ', data['que'][15])

In [9]:
que

'Is anyone there '

In [10]:
que = que.lower().split()

In [11]:
que

['is', 'anyone', 'there']

In [12]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [13]:
from nltk.corpus import stopwords

In [14]:
que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]

In [15]:
que

['anyon']

In [21]:
data.shape[0]

38

In [22]:
corpus = list()
for i in range(0, data.shape[0]):
    que = re.sub('[^a-zA-Z]', ' ', data['que'][i])
    que = que.lower().split()
    que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]
    que = ' '.join(que)
    corpus.append(que)

In [23]:
len(corpus)

38

In [24]:
corpus

['name',
 'hey',
 'servic firm provid',
 'cya',
 'servic',
 'time avail',
 'good day',
 'loaction',
 '',
 'email id',
 'hour oper',
 'bye',
 'offic time',
 'good day',
 'leav',
 'anyon',
 'time oper',
 'meet',
 'hi',
 'hy',
 'provid',
 'locat offic',
 'mobil number',
 'guy open',
 'call',
 'see later',
 'name',
 'meet',
 'address',
 'meet',
 'what',
 'may contact',
 '',
 'work time',
 'hour',
 'goodby',
 'phone number',
 'hello']

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X = cv.fit_transform(corpus).toarray()
y = data.iloc[:, 1].values

In [26]:
X.shape

(38, 38)

In [27]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [28]:
s = "how are you?"
que = re.sub('[^a-zA-Z]', ' ', s)
que = que.lower().split()
que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]
que = ' '.join(que)
test = cv.transform([que]).toarray()
ans = classifier.predict(test)
print(ans)

['greeting']


In [29]:
import json

with open('response.json') as f:
    responses = json.load(f)

In [30]:
import random
for item in responses['respones']:
    if item['tag'] == 'greeting':
        l = random.randint(0,(len(item['responses'])-1))
        print(item['responses'][l])

Hello!


In [31]:
def create_response_sci(que):
    if que == "quit":
        print("Bye see you soon")
        return False
    que = re.sub('[^a-zA-Z]', ' ', que)
    que = que.lower().split()
    que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]
    que = ' '.join(que)
    test = cv.transform([que]).toarray()
    ans = classifier.predict(test)
    print("samvadCAbot--> ",end=" ")
    a = False
    for item in responses['respones']:
        if item['tag'] == ans:
            l = random.randint(0,(len(item['responses'])-1))
            a = True
            print(item['responses'][l])
    if not a:
        print("Sorry couldn't get you,please repeat")
        
    return True

In [32]:
a = "Where can i meet you"
create_response_sci(a)

samvadCAbot-->  Our office is loacted at https://tinyurl.com/samvadcaAddress 
 (Or) Contact us at samvadca@gmail.com


True

In [36]:
def chat_sci():
    a = True
    print("samvadCAbot--> Hello there!How can I help you")
    while a:
        inp = input("Enter (type quit to exit)")
        a = create_response_sci(inp)

In [37]:
chat_sci()

samvadCAbot--> Hello there!How can I help you
Enter (type quit to exit)Hello
samvadCAbot-->  Hi there, how can I help?
Enter (type quit to exit)How are you
samvadCAbot-->  Hello!
Enter (type quit to exit)how are you
samvadCAbot-->  Hi there, how can I help?
Enter (type quit to exit)Is anyone up there
samvadCAbot-->  Hello!
Enter (type quit to exit)quit
Bye see you soon


<h3>Now using the keras classifier for better classification</h3> 
    <p>I will be using Keras Sequential model</p>

In [38]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [39]:
data.label.nunique()

7

In [40]:
from sklearn.preprocessing import LabelEncoder
lb_y = LabelEncoder()
Y = lb_y.fit_transform(y)

In [41]:
y_ohe = keras.utils.np_utils.to_categorical(Y,num_classes=7)

In [46]:
classifier_nn = Sequential()

In [47]:
classifier_nn.add(Dense(21,kernel_initializer="uniform",activation="relu",input_dim=X.shape[1]))
classifier_nn.add(Dense(14, kernel_initializer = 'uniform', activation = 'relu'))
classifier_nn.add(Dense(units = 7, kernel_initializer = 'uniform', activation = 'softmax'))

In [48]:
classifier_nn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [49]:
classifier_nn.fit(X,y_ohe, batch_size = 16, epochs = 500)

Epoch 1/500
38/38 [==============================] - 0s 2ms/step - loss: 1.9461 - accuracy: 0.1579
Epoch 2/500
38/38 [==============================] - 0s 103us/step - loss: 1.9455 - accuracy: 0.1842
Epoch 3/500
38/38 [==============================] - 0s 105us/step - loss: 1.9450 - accuracy: 0.1842
Epoch 4/500
38/38 [==============================] - 0s 180us/step - loss: 1.9446 - accuracy: 0.2105
Epoch 5/500
38/38 [==============================] - 0s 238us/step - loss: 1.9441 - accuracy: 0.3684
Epoch 6/500
38/38 [==============================] - 0s 136us/step - loss: 1.9436 - accuracy: 0.3947
Epoch 7/500
38/38 [==============================] - 0s 325us/step - loss: 1.9432 - accuracy: 0.3684
Epoch 8/500
38/38 [==============================] - 0s 299us/step - loss: 1.9425 - accuracy: 0.3684
Epoch 9/500
38/38 [==============================] - 0s 292us/step - loss: 1.9420 - accuracy: 0.3684
Epoch 10/500
38/38 [==============================] - 0s 161us/step - loss: 1.9412 - accuracy

38/38 [==============================] - 0s 178us/step - loss: 1.1533 - accuracy: 0.5263
Epoch 82/500
38/38 [==============================] - 0s 260us/step - loss: 1.1412 - accuracy: 0.5526
Epoch 83/500
38/38 [==============================] - 0s 214us/step - loss: 1.1286 - accuracy: 0.5789
Epoch 84/500
38/38 [==============================] - 0s 242us/step - loss: 1.1163 - accuracy: 0.5789
Epoch 85/500
38/38 [==============================] - 0s 233us/step - loss: 1.1044 - accuracy: 0.5789
Epoch 86/500
38/38 [==============================] - 0s 274us/step - loss: 1.0926 - accuracy: 0.5789
Epoch 87/500
38/38 [==============================] - 0s 177us/step - loss: 1.0800 - accuracy: 0.6053
Epoch 88/500
38/38 [==============================] - 0s 237us/step - loss: 1.0681 - accuracy: 0.6053
Epoch 89/500
38/38 [==============================] - 0s 296us/step - loss: 1.0557 - accuracy: 0.6579
Epoch 90/500
38/38 [==============================] - 0s 285us/step - loss: 1.0433 - accuracy: 

38/38 [==============================] - 0s 121us/step - loss: 0.5075 - accuracy: 0.8158
Epoch 161/500
38/38 [==============================] - 0s 168us/step - loss: 0.5027 - accuracy: 0.8158
Epoch 162/500
38/38 [==============================] - 0s 208us/step - loss: 0.4986 - accuracy: 0.8158
Epoch 163/500
38/38 [==============================] - 0s 134us/step - loss: 0.4935 - accuracy: 0.8158
Epoch 164/500
38/38 [==============================] - 0s 184us/step - loss: 0.4892 - accuracy: 0.8158
Epoch 165/500
38/38 [==============================] - 0s 143us/step - loss: 0.4850 - accuracy: 0.8158
Epoch 166/500
38/38 [==============================] - 0s 220us/step - loss: 0.4801 - accuracy: 0.8158
Epoch 167/500
38/38 [==============================] - 0s 140us/step - loss: 0.4762 - accuracy: 0.8158
Epoch 168/500
38/38 [==============================] - 0s 254us/step - loss: 0.4716 - accuracy: 0.8158
Epoch 169/500
38/38 [==============================] - 0s 125us/step - loss: 0.4677 - a

38/38 [==============================] - 0s 249us/step - loss: 0.1591 - accuracy: 0.9211
Epoch 319/500
38/38 [==============================] - 0s 148us/step - loss: 0.1593 - accuracy: 0.9211
Epoch 320/500
38/38 [==============================] - 0s 221us/step - loss: 0.1588 - accuracy: 0.9211
Epoch 321/500
38/38 [==============================] - 0s 243us/step - loss: 0.1598 - accuracy: 0.9211
Epoch 322/500
38/38 [==============================] - 0s 167us/step - loss: 0.1600 - accuracy: 0.9211
Epoch 323/500
38/38 [==============================] - 0s 239us/step - loss: 0.1597 - accuracy: 0.9211
Epoch 324/500
38/38 [==============================] - 0s 238us/step - loss: 0.1582 - accuracy: 0.9211
Epoch 325/500
38/38 [==============================] - 0s 140us/step - loss: 0.1575 - accuracy: 0.9211
Epoch 326/500
38/38 [==============================] - 0s 278us/step - loss: 0.1581 - accuracy: 0.9211
Epoch 327/500
38/38 [==============================] - 0s 165us/step - loss: 0.1569 - a

38/38 [==============================] - 0s 198us/step - loss: 0.1371 - accuracy: 0.9211
Epoch 477/500
38/38 [==============================] - 0s 164us/step - loss: 0.1371 - accuracy: 0.9211
Epoch 478/500
38/38 [==============================] - 0s 249us/step - loss: 0.1376 - accuracy: 0.9211
Epoch 479/500
38/38 [==============================] - 0s 131us/step - loss: 0.1372 - accuracy: 0.9211
Epoch 480/500
38/38 [==============================] - 0s 241us/step - loss: 0.1376 - accuracy: 0.8947
Epoch 481/500
38/38 [==============================] - 0s 168us/step - loss: 0.1383 - accuracy: 0.8947
Epoch 482/500
38/38 [==============================] - 0s 209us/step - loss: 0.1368 - accuracy: 0.9211
Epoch 483/500
38/38 [==============================] - 0s 287us/step - loss: 0.1371 - accuracy: 0.8684
Epoch 484/500
38/38 [==============================] - 0s 182us/step - loss: 0.1372 - accuracy: 0.9211
Epoch 485/500
38/38 [==============================] - 0s 196us/step - loss: 0.1361 - a

In [52]:
Y

array([5, 3, 6, 2, 6, 4, 3, 0, 3, 1, 4, 2, 4, 2, 2, 3, 4, 0, 3, 3, 6, 0,
       1, 4, 5, 2, 5, 0, 0, 1, 3, 1, 6, 4, 4, 2, 1, 3])

In [85]:
s = "Hello"
que = re.sub('[^a-zA-Z]', ' ', s)
que = que.lower().split()
que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]
que = ' '.join(que)
test = cv.transform([que]).toarray()
ans = classifier_nn.predict_classes(test)

In [86]:
ans

array([3])

In [55]:
np.argmax(ans, axis=None, out=None)

0

In [90]:
lb_y.inverse_transform(ans)[0]

'greeting'

In [60]:
def create_response(que):
    if que == "quit" or que == "exit":
        print("Bye see you soon")
        return False
    que = re.sub('[^a-zA-Z]', ' ', que)
    que = que.lower().split()
    que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]
    que = ' '.join(que)
    test = cv.transform([que]).toarray()
    ans = classifier_nn.predict_classes(test)
    ans = lb_y.inverse_transform(ans)[0]
    print("samvadCAbot--> ",end=" ")
    a = False
    for item in responses['respones']:
        if item['tag'] == ans:
            l = random.randint(0,(len(item['responses'])-1))
            a = True
            print(item['responses'][l])
    if not a:
        print("Sorry couldn't get you,please repeat")
        
    return True

In [61]:
def chat():
    a = True
    print("samvadCAbot--> Hello there!How can I help you")
    while a:
        inp = input("Enter (type quit to exit)")
        a = create_response(inp)

In [59]:
chat()

samvadCAbot--> Hello there!How can I help you
Enter (type quit to exit)hi
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)hello
samvadCAbot-->  Hello!
Enter (type quit to exit)hy
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)hey
samvadCAbot-->  Hello!
Enter (type quit to exit)how are you
samvadCAbot-->  Hi there, how can I help?
Enter (type quit to exit)whats up
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)how are you
samvadCAbot-->  Hello!
Enter (type quit to exit)how are you
samvadCAbot-->  Hello!
Enter (type quit to exit)how are you
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)name
samvadCAbot-->  You can call me Samvad_CA.
Enter (type quit to exit)who is albert ein
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)who is albert eninsten
samvadCAbot-->  Hello!
Enter (type quit to exit)exit
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)quit
Bye see you soon


<p><b>We will create a method to get the probability of the classes question belong too.And then we will set a thresold for getting better result</b></p>

In [98]:
s = "Hello"
que = re.sub('[^a-zA-Z]', ' ', s)
que = que.lower().split()
que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]
que = ' '.join(que)
test = cv.transform([que]).toarray()
ans = classifier_nn.predict_proba(test)

In [99]:
ans

array([[1.2660275e-05, 2.8450952e-03, 2.9213747e-05, 9.9269217e-01,
        1.6307300e-08, 1.1171482e-03, 3.3038054e-03]], dtype=float32)

In [100]:
ans = np.argmax(ans)

In [104]:
np.asarray([ans])

array([3])

In [106]:
lb_y.inverse_transform(np.asarray([ans]))[0]

'greeting'

In [111]:
def create_response_prob(que):
    if que == "quit" or que == "exit":
        print("Bye see you soon")
        return False
    print("samvadCAbot--> ",end=" ")
    #Preprocess the text
    que = re.sub('[^a-zA-Z]', ' ', que)
    que = que.lower().split()
    que = [ps.stem(word) for word in que if not word in set(stopwords.words('english'))]
    que = ' '.join(que)
    test = cv.transform([que]).toarray()
    #get the proprobaliltiy of each class
    ans = classifier_nn.predict_proba(test)
    a = False
    if np.argmax(ans) > 0.7:
        ans = np.argmax(ans)
        ans= lb_y.inverse_transform(np.asarray([ans]))[0]
        for item in responses['respones']:
            if item['tag'] == ans:
                l = random.randint(0,(len(item['responses'])-1))
                a = True
                print(item['responses'][l])
    if not a:
        print("Sorry couldn't get you,please repeat")
        
    return True

In [112]:
def chat_prob():
    a = True
    print("samvadCAbot--> Hello there!How can I help you")
    while a:
        inp = input("Enter (type quit to exit)")
        a = create_response_prob(inp)

In [113]:
chat_prob()

samvadCAbot--> Hello there!How can I help you
Enter (type quit to exit)wiesle hagi maro
samvadCAbot-->  Hello!
Enter (type quit to exit)how you doing
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)na ji na
samvadCAbot-->  Good to see you again!
Enter (type quit to exit)quit
Bye see you soon
